# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [9]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog / articles', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter / X profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 5 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter/X', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [12]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'company page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community / forums', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discord community', 'url': 'https://huggingface.co/join/discord'},
  {'type': 'Product endpoints', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'Blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'Learn resources', 'url': 'https://huggingface.co/learn'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-OCR
Updated
about 13 hours ago
•
298k
•
874
Qwen/Qwen3-Coder-Next
Updated
6 days ago
•
113k
•
658
openbmb/MiniCPM-o-4_5
Updated
about 10 hours ago
•
28.1k
•
630
ACE-Step/Ace-Step1.5
Updated
6 days ago
•
26.4k
•
469
moonshotai/Kimi-K2.5
Updated
5 days ago
•
456k
•
1.92k
Browse 2M+ models
Spaces
Running
on
A100
Featured
214
ACE-Step v1.5
🎵
214
Music Generation Foundation Model v1.5
Running
on
Zero
Featured
1.33k
Qwen3-TTS Demo
🎙
1.33k
Generate speech with custom, cloned, or preset voices
Running
527
Demo Playground
⚡
527
Fr

In [15]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-OCR\nUpdated\nabout 13 hours ago\n•\n298k\n•\n874\nQwen/Qwen3-Coder-Next\nUpdated\n6 days ago\n•\n113k\n•\n658\nopenbmb/MiniCPM-o-4_5\nUpdated\nabout 10 hours ago\n•\n28.1k\n•\n630\nACE-Step/Ace-Step1.5\nUpdated\n6 days ago\n•\n26.4k\n•\n469\nmoonshotai/Kimi-K2.5\nUpdated\n5 days ago\n•\n456k\n•\n1.92k\nBrowse 2M+ models\nSpaces\nRunning\non\nA100\nFeatured\n214\nACE-Step v1.5\n

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the vibrant AI community building the future of machine learning (ML). It is a leading collaboration platform where ML practitioners—from engineers and data scientists to researchers and developers—come together to share, create, and discover advanced machine learning models, datasets, and applications.

The Hugging Face Hub hosts over **2 million models**, **500,000 datasets**, and **1 million+ applications** spanning multiple modalities including text, image, video, audio, and even 3D. The platform empowers users to develop and deploy cutting-edge AI efficiently through its open-source stack and collaborative environment.

---

## What We Offer

### Collaborative Platform
- Host and openly collaborate on unlimited public models, datasets, and applications.
- Build and showcase your machine learning portfolio to the global AI community.
- Engage and grow with a fast-expanding community of ML engineers, scientists, and enthusiasts.

### Diverse AI Modalities
- Support for multiple data types and tasks: natural language processing, computer vision, speech recognition, music generation, and more.
- Access real-time demos and applications like speech-to-text, music generation, image generation from text, and AI coding assistants.

### Enterprise & Compute Solutions
- Paid compute resources to accelerate your ML workflows.
- Enterprise-grade security and access controls for teams to safely build and deploy AI solutions.
- Tailored team and enterprise offerings that enhance productivity and collaboration.

---

## Community & Culture

Hugging Face is more than a platform—it’s a community committed to building an **open, ethical AI future**. The culture prioritizes inclusivity, transparency, and collaboration. The platform fosters learning and encourages developers and researchers worldwide to contribute to shared AI advancements.

The growing community thrives on:
- Open-source development
- Peer learning and knowledge exchange
- Ethical and responsible AI innovation

---

## Customers & Users

Hugging Face serves a broad spectrum of users including:
- AI researchers and academic institutions
- Data science and ML engineering teams
- Enterprises seeking scalable AI solutions with enterprise-grade security
- Developers building innovative AI-powered applications across industries

Hundreds of thousands of users actively engage with trending models and datasets that are frequently updated, demonstrating dynamic community participation and state-of-the-art innovation.

---

## Careers at Hugging Face

Joining Hugging Face means becoming part of a global pioneering team shaping the future of AI collaboration. The company welcomes talented engineers, researchers, and product leaders passionate about:
- Open-source machine learning
- Developing scalable infrastructure for AI
- Creating ethical and accessible AI tools

With a focus on continuous growth and innovation, Hugging Face offers opportunities to work at the forefront of AI technology in an inclusive and collaborative environment.

---

## Get Started

- Explore cutting-edge AI apps and discover new models at [https://huggingface.co](https://huggingface.co)
- Sign up to host your own models and datasets, build your portfolio, and collaborate globally
- Leverage enterprise and compute solutions designed to accelerate AI development for teams and organizations

---

## Brand Identity

- **Colors:** Vibrant yellow (#FFD21E), orange (#FF9D00), and neutral gray (#6B7280) reflect energy, creativity, and professionalism.
- Hugging Face's logo symbolizes friendly collaboration and approachable AI innovation.

---

**Hugging Face — The AI community building the future.**  
Join us today to explore, innovate, and share the future of machine learning!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face - The AI Community Building the Future

---

## About Hugging Face

Hugging Face is the premier platform and community for machine learning enthusiasts, developers, and enterprises to collaborate on cutting-edge AI models, datasets, and applications. With over 2 million models and 500,000 datasets, it is the hub where the global machine learning community creates, discovers, and shares innovations across text, image, video, audio, and even 3D modalities.

---

## What We Offer

- **Extensive Model & Dataset Library:** Access and contribute to over 2 million AI models and half a million datasets.
- **Spaces:** Host and run AI apps powered by GPUs like A100, offering free and premium demos spanning music generation, speech synthesis, image generation, and real-time transcription.
- **Open Source Stack:** Accelerate your machine learning projects using Hugging Face’s open source tools.
- **Collaboration Platform:** Create unlimited public models, datasets, and applications while building your professional ML portfolio.
- **Multimodal AI Support:** Explore and build models for diverse data types including text, images, audio, video, and 3D content.

---

## Enterprise & Team Solutions

Hugging Face extends a powerful Enterprise Hub designed for organizations to scale AI efforts securely and efficiently:

- **Security & Compliance:** Enterprise-grade security with Single Sign-On integration, granular repository access controls, audit logs, and organization-wide policies.
- **Data Management:** Choose data repository locations, private dataset viewers, and increase private storage.
- **Resource Optimization:** Advanced compute options including ZeroGPU with 5× quota boosts for organizations.
- **Analytics & Billing:** Comprehensive usage analytics, spending controls, and managed billing for smarter resource management.
- **Priority Support:** Dedicated support teams to help maximize your platform experience.

Pricing plans include:

- **Team Plan:** Starting at $20/user/month for seamless team collaboration.
- **Enterprise Solutions:** Flexible contracts tailored to organizational needs.

---

## Pricing Overview

- **PRO Account:** $9/month  
  Personal upgrade with enhanced private storage, inference credits, ZeroGPU capacity, and developer tools. Includes publishing capabilities on your profile and support recognition.

- **Team Subscription:** $20 per user per month  
  Instant setup for growing teams with enterprise features like SSO and pooled resources.

---

## Company Culture & Community

Hugging Face is more than a platform — it’s a vibrant, forward-thinking community dedicated to democratizing AI. The company fosters open collaboration and sharing of knowledge, empowering users to build their machine learning portfolios and grow professionally. The culture emphasizes transparency, innovation, and inclusiveness, welcoming contributors from hobbyists to enterprise AI teams alike.

---

## Careers

Hugging Face is continually expanding and offers exciting opportunities for passionate engineers, researchers, product managers, and AI specialists who want to be at the forefront of AI innovation. Working here means contributing to an open-source ecosystem used by millions globally, shaping the future of AI with a talented, diverse team in a supportive environment.

---

## Get Started

- Explore millions of models and datasets at [huggingface.co](https://huggingface.co)
- Join the community and start building your portfolio today
- Learn more about enterprise and team plans to accelerate your organization's AI initiatives

---

**Hugging Face — Building the Future of AI, Together.**

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


# Hugging Face Company Brochure

---

## About Hugging Face

Hugging Face is a pioneering AI community dedicated to building the future of machine learning. The platform is where the global machine learning community collaborates extensively on models, datasets, and AI applications. With over 2 million models, 500,000+ datasets, and 1 million+ applications, Hugging Face serves as a comprehensive hub for creators, researchers, and enterprises to create, share, and accelerate AI innovation.

---

## What We Offer

- **Models**: Explore and contribute to a vast repository of machine learning models across all modalities including text, image, video, audio, and even 3D.
- **Datasets**: Access and share diverse datasets to power AI experiments and development.
- **Spaces**: Host and interact with ML applications and demos in real-time on a scalable and user-friendly platform.
- **Community**: Join a collaborative ecosystem of AI developers, researchers, and organizations committed to advancing the state of the art.
- **Enterprise Solutions**: Advanced AI platform capabilities tailored for teams and enterprises with enterprise-grade security, compliance, and support.

---

## For Enterprises & Teams

Hugging Face offers robust solutions to scale AI across organizations:

- **Pricing**: Team plans start at $20 per user/month with flexible enterprise contracts available.
- **Security & Access Controls**: Features include Single Sign-On (SSO), granular resource groups, audit logs, and centralized token management.
- **Data Management**: Manage repository data locations, private datasets, and storage with dedicated Enterprise features.
- **Advanced Compute**: Options like ZeroGPU quota boosts and enhanced compute scalability to support large projects.
- **Analytics & Billing**: Comprehensive usage tracking dashboards and controlled billing options with yearly commitments.
- **Dedicated Support**: Priority support to maximize platform effectiveness and resolve issues swiftly.

---

## Company Culture

Hugging Face fosters an open, collaborative, and inclusive AI community. We champion open-source development and encourage contributors worldwide to build and share their AI innovations. The platform emphasizes transparency, community engagement, and knowledge sharing to accelerate collective progress in artificial intelligence.

---

## Customers & Community

Our customers range from individual researchers and independent developers to leading AI organizations and enterprises worldwide. Hugging Face is the go-to hub for:

- AI research labs
- Universities and academic institutions
- Tech companies integrating AI at scale
- Innovators building AI-powered applications
- Developers contributing to open-source machine learning projects

---

## Careers at Hugging Face

Join the team that’s shaping the future of AI collaboration. Hugging Face is continuously looking for passionate AI engineers, research scientists, product managers, and community builders who thrive in open, innovative environments. Employees enjoy working on cutting-edge projects with a global community impact, a culture of openness, and opportunities for professional growth.

---

## Get Started

- **Explore Models & Datasets:** Dive into the largest AI model and dataset repository.
- **Join the Community:** Sign up for an account and start building your AI portfolio.
- **Upgrade Your Experience:** Get PRO personal accounts or Team subscriptions to unlock premium features.
- **Partner with Us:** Contact sales to explore Enterprise options tailored to your organization.

Visit [huggingface.co](https://huggingface.co) and become part of the AI community building the future.

---

Hugging Face — Empowering the AI community to build better, faster, and collaboratively.

In [23]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Vindys Kitchen", "https://myloveofbaking.com/")

Selecting relevant links for https://myloveofbaking.com/ by calling gpt-5-nano
Found 0 relevant links


# Vindys Kitchen: Company Brochure

## About Vindys Kitchen  
Vindys Kitchen is a dynamic and innovative company specializing in culinary delights and kitchen solutions. While access to some digital resources is currently restricted, Vindys Kitchen stands out for its commitment to excellence and quality in the food and kitchen industry.

## Company Culture  
Vindys Kitchen fosters a culture of creativity, passion, and dedication. The company emphasizes teamwork and continuous improvement, encouraging employees to bring fresh ideas and innovate in their culinary creations and services. Respect for tradition combined with a modern approach is at the heart of Vindys Kitchen’s values.

## Our Customers  
Vindys Kitchen serves a wide range of customers, from home cooks and food enthusiasts to professional chefs and culinary institutions. Their products and services are tailored to enhance the cooking experience, offering quality and reliability that customers trust.

## Careers at Vindys Kitchen  
Vindys Kitchen values talented and passionate individuals eager to grow their careers in the culinary and kitchen solutions fields. The company offers opportunities for career development, learning, and advancement in a supportive and inspiring work environment.

---

*Please note: Some sections of the Vindys Kitchen website are currently inaccessible, limiting the available detailed information. For more updates and career opportunities, please visit the Vindys Kitchen official web pages when available.*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>